# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [2]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode
import pandas as pd

In [3]:
def getNGrams(words,n):
    ngrams = {}
    for word in words:
        # word="$"*(n-1)+word+"$"*(n-1)
        if len(word) <n:
            continue
        for i in range(len(word)-n+1):
            ngram = word[i:i+n]
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram]+=1
    return ngrams



def load_model(model_path):
    model = RNN_model(target_dim=26, hidden_units=16)
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model



In [4]:
word=["abc"]
# guessed=list(set(word)-set("_"))
print(getNGrams(word,1))

{'a': 1, 'b': 1, 'c': 1}


In [5]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None,model_path='model.pth',n_gram=2):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        full_dictionary_location="words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.freq_by_length = self.init_df(self.full_dictionary)
        self.n_gram = self.init_n_gram(n_gram)
        self.current_dictionary = []
        self.history_condition = []
        self.model = load_model(model_path)
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")

        words = text_file.read().splitlines()
        self.ngrams=[getNGrams(words,i) for i in range(1,6)]

        # self.current_dictionary = []
    def buildDict(self):
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        
        self.ngrams=[getNGrams(words,i) for i in range(1,6)]
        
    def ngramprob(self,word,letter ):
        # print("________________________: word:",word,"letter:",letter)
        denominator=0
        underscroll_location=word.find("_")
        # if len(word)>2:
        #     print("word:",word,"letter:",letter)
        #     for l in string.ascii_lowercase:
        #         print("l:",l)
        #         print(self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]])

        for l in string.ascii_lowercase:
            
            try:
                denominator+=self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]]
            except:
                pass
        # print("____________:",word)
        # if len(word)>3:
        #     print("word with length >3:",word)
        # if word=="a_s":
        #     print("letter:",letter)
        #     print("denominator:",denominator)
        #     print("numerator:",self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]])
        return self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]]/denominator

        
    # def guess(self, word): # word input example: "_ p p _ e "
    #     ###############################################
    #     # Replace with your own "guess" function here #
    #     ###############################################

    #     # clean the word so that we strip away the space characters
    #     # replace "_" with "." as "." indicates any character in regular expressions
    #     # clean_word = word[::2].replace("_",".")
    #     # guessed=list(set(word)-set("_"))
    #     word=word[::2]
    #     available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
    #     probs=[]
    #     contributions=[0,0,0,0,0]
    #     for char in available:
    #         char_prob=0
    #         for i in range(len(word)):
    #             if word[i]!="_":
    #                 continue
    #             # find the longest continious string without any "_"

    #             # 4gram
    #             try:
    #                 if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_":
    #                     contributions[3]+=self.ngramprob(word[i-3:i+1],char)
    #                     char_prob+=self.ngramprob(word[i-3:i+1],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_":
    #                     contributions[3]+=self.ngramprob(word[i-2:i+2],char)
    #                     char_prob+=self.ngramprob(word[i-2:i+2],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_":
    #                     contributions[3]+=self.ngramprob(word[i-1:i+3],char)
    #                     char_prob+=self.ngramprob(word[i-1:i+3],char)
    #             except:
    #                 pass
    #             try: 
    #                 if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_":
    #                     contributions[3]+=self.ngramprob(word[i:i+4],char)
    #                     char_prob+=self.ngramprob(word[i:i+4],char) 
    #             except:
    #                 pass
    #             # 3gram
    #             try:
    #                 if word[i-1]!="_" and word[i-2]!="_":
    #                     contributions[2]+=self.ngramprob(word[i-2:i+1],char)
    #                     char_prob+=self.ngramprob(word[i-2:i+1],char)
    #             except:
    #                 pass
    #             try:
    #                 # a_b
    #                 if word[i+1]!="_" and word[i-1]!="_":
    #                     contributions[2]+=self.ngramprob(word[i-1:i+2],char)
    #                     char_prob+=self.ngramprob(word[i-1:i+2],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i+2]!="_":
    #                     contributions[2]+=self.ngramprob(word[i:i+3],char)
    #                     char_prob+=self.ngramprob(word[i:i+3],char)
    #             except:
    #                 pass
    #                 # 2gram
    #             try:
    #                 if word[i-1]!="_":
    #                     contributions[1]+=self.ngramprob(word[i-1:i+1],char)
    #                     char_prob+=self.ngramprob(word[i-1:i+1],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_":
    #                     contributions[1]+=self.ngramprob(word[i:i+2],char)
    #                     char_prob+=self.ngramprob(word[i:i+2],char)
    #             except:
    #                 pass
    #             # 5 gram
    #             try:
    #                 if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" :
    #                     contributions[4]+=self.ngramprob(word[i-4:i+1],char)
    #                     char_prob+=self.ngramprob(word[i-4:i+1],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" :
    #                     contributions[4]+=self.ngramprob(word[i-3:i+2],char)
    #                     char_prob+=self.ngramprob(word[i-3:i+2],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_" and word[i-2]!="_" :
    #                     contributions[4]+=self.ngramprob(word[i-2:i+3],char)
    #                     char_prob+=self.ngramprob(word[i-2:i+3],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i-1]!="_" :
    #                     contributions[4]+=self.ngramprob(word[i-1:i+4],char)
    #                     char_prob+=self.ngramprob(word[i-1:i+4],char)
    #             except:
    #                 pass
    #             try:
    #                 if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" :
    #                     contributions[4]+=self.ngramprob(word[i:i+5],char)
    #                     char_prob+=self.ngramprob(word[i:i+5],char)
    #             except:
    #                 pass
                

    #             # 1gram
    #             char_prob+=self.ngramprob(word[i:i+1],char)
    #             contributions[0]+=self.ngramprob(word[i:i+1],char)
    #         probs.append(char_prob)
    #     # create sorted dictionary of probabilities
    #     probs_dict=dict(zip(available,probs))
    #     probs_sorted=sorted(probs_dict.items(), key=lambda x: x[1], reverse=True)
    #     # iterate on probs_sorted
    #     for i in probs_sorted:
    #         # round the probability to 3 decimal places
    #         # i[1]=round()
    #         print(i[0],round(i[1],2),end=", ")
    #     # for i in range(len(probs)):
    #     #     # print upto 2 decimal places
    #     #     # probs[i]=round(probs[i],2)
    #     #     print(probs_sorted[i],round(probs_sorted[i],2),end=" ")
    #     print()
    #     print("contributions:",contributions)
    #     return available[probs.index(max(probs))]
            
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
    
    def find_by_gram(self, all_gram, pre=None, suff=None):
        selected_gram = []
        for key, val in all_gram.items():
            if (pre is not None) and (key[0] == pre):
                selected_gram.append((key[1], val))
            if (suff is not None) and (key[1] == suff):
                selected_gram.append((key[0], val))

        res = {}
        for letter, freq in selected_gram:
            if letter not in res:
                res[letter] = freq
            else:
                res[letter] += freq
        final_res = [(key, val) for key, val in res.items()]
        return sorted(final_res, key=lambda x: x[1], reverse=True)

    def gen_n_gram(self, word, n):
        n_gram = []
        for i in range(n, len(word)+1):
            if word[i-n:i] not in n_gram:
                n_gram.append(word[i-n:i])
        return n_gram

    def init_n_gram(self, n):
        n_gram = {-1:[]}
        for word in self.full_dictionary:
            single_word_gram = self.gen_n_gram(word, n)
            if len(word) not in n_gram:
                n_gram[len(word)] = single_word_gram
            else:
                n_gram[len(word)].extend(single_word_gram)
            n_gram[-1].extend(single_word_gram)
        res = {}
        for key in n_gram.keys():
            res[key] = collections.Counter(n_gram[key])
        return res

    def freq_from_df(self, df):
        key, cnt = np.unique(df.values, return_counts=True)
        freq = [(k, val) for k, val in zip(key, cnt)]
        return sorted(freq, key=lambda x: x[1], reverse=True)

    def update_df(self, df, condition):
        """
        :param df: dataframe
        each column is one location of a word
        each row is a word
        :param condition: dictionary
        key is letter
        value is which index does this letter appear
        means we only select the words which has letter <value> at index <key>
        note that we don't select words that has letter <value> at other index
        e.g. if condition = {1:'a'}, then "app" is selected while "aha" not
        :return:
        df: updated dataframe
        """
        if len(condition) == 0:
            return df

        for letter, idx in condition.items():
            # find rows satisfy
            # 1. corresponding column == val
            # 2. all the other column != val
            query = ""
            for i in range(df.shape[1]):
                col = df.columns.values[i]
                if i in idx:
                    query += "{} == '{}' and ".format(col, letter)
                else:
                    query += "{} != '{}' and ".format(col, letter)
            query = query[:-5]
            new_df = df.query(query)
            df = new_df.copy()
            del new_df
        return df

    def init_df(self, dictionary):
        """
        use words list to generate dictionary frequency
        each key is word length
        each value is a dataframe with column is location of each length
        """
        group_by_length = collections.defaultdict(list)
        for word in dictionary:
            group_by_length[len(word)].append(word)

        res = {}
        for key in group_by_length.keys():
            word_list = group_by_length[key]
            tmp = pd.DataFrame([list(word) for word in word_list])
            tmp.columns = [chr(i + 97) for i in range(tmp.shape[1])]
            res[key] = tmp
        return res

    def gen_condition(self, word):
        tmp = {i: word[i] for i in range(len(word)) if word[i] != "_"}
        condition = {}
        for key, val in tmp.items():
            if val not in condition:
                condition[val] = [key]
            else:
                condition[val].append(key)
        return condition

    def encode_obscure_words(self, word):
        word_idx = [ord(i) - 97 if i != "_" else 26 for i in word]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word_idx):
            obscured_word[i, j] = 1
        return obscured_word

    def guess(self, word):  # word input example: "_ p p _ e "

        # divided word group by word length
        all_words = self.freq_by_length[len(word)]
        all_gram = self.n_gram[-1]
        # all_gram = self.n_gram[len(word)]

        # first guess by letter frequency in each word group
        new_condition = self.gen_condition(word)

        if len(self.history_condition) != 0 and new_condition != self.history_condition[-1]:
            self.history_condition.append(new_condition)

        all_words = self.update_df(all_words, new_condition)
        freq = self.freq_from_df(all_words)
        for i in range(len(freq)):
            if freq[i][0] not in self.guessed_letters:
                return freq[i][0]

        # if we run out of letters, use 2-gram to predict
        for i in range(len(word)):
            if word[i] == "_":  # this is where we should apply 2-gram
                if (i == 0) or (word[i-1] == "_"):
                    guess = self.find_by_gram(all_gram, pre=None, suff=word[i+1])
                elif (i == len(word) - 1) or (word[i+1] == "_"):
                    guess = self.find_by_gram(all_gram, pre=word[i-1], suff=None)
                else:
                    guess = self.find_by_gram(all_gram, pre=word[i-1], suff=word[i+1])
                break

        for i in range(len(guess)):
            if guess[i][0] not in self.guessed_letters:
                return guess[i][0]
        # if we run out of 2-gram, use LSTM model to predict
        # the benefit of LSTM model is to add more uncertainty to the prediction
        guessed_multi_hot = np.zeros(26, dtype=np.float32)
        for letter in self.guessed_letters:
            idx = ord(letter) - 97
            guessed_multi_hot[idx] = 1.0

        obscure_words = self.encode_obscure_words(word)
        obscure_words = np.asarray(obscure_words)
        guessed_multi_hot = np.asarray(guessed_multi_hot)
        obscure_words = torch.from_numpy(obscure_words)
        guessed_multi_hot = torch.from_numpy(guessed_multi_hot)
        out = self.model(obscure_words, guessed_multi_hot)
        guess = torch.argmax(out, dim=2).item()
        guess = chr(guess + 97)
        return guess

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def get_current_word(self):
        """
        combine target word and guessed letters to generate obscured word
        """
        word_seen = [letter if letter in self.guessed_letters else "_" for letter in self.target_word]
        return word_seen              
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        # self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            word=word.split(" ")[:-1]
            print("Word:",word)
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
                    word=word.split(" ")[:-1]
                    
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    def offline(self,wordtobeguessed):
        self.guessed_letters=[]
        tries_remains=6
        word = "_ " * len(wordtobeguessed)
        assert len(word) == len(wordtobeguessed) *2
        while tries_remains>0:
                # get guessed letter from user code
            guess_letter = self.guess(word)
            flag=False
            for index in range(len(wordtobeguessed)):
                if wordtobeguessed[index]==guess_letter:
                    word=word[:2*index]+guess_letter+word[2*index+1:]
                    flag=True
            if flag==False:
                tries_remains-=1
                print("Wrong guess, you have",tries_remains,"tries left")
            if "_" not in word:
                print("You won!")
                break
            # append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            # time.sleep(0.5)
            print("Guessed letter: {0}".format(guess_letter))
            print("Current word: {0}".format(word))
            
            # tries_remains -=1
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

    # def start_game(self, num_lives=6, verbose=True):

    #     self.target_word = input("please enter a word for the computer to guess:")
    #     # reset guessed letters to empty set and current plausible dictionary to the full dictionary
    #     self.guessed_letters = []
    #     self.current_dictionary = self.full_dictionary
    #     tries_remains = num_lives

    #     word_seen = self.get_current_word()
    #     if verbose:
    #         print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word_seen))

    #     while tries_remains > 0:
    #         # get guessed letter from user code
    #         guess_letter = self.guess(word_seen)

    #         # append guessed letter to guessed letters field in hangman object
    #         self.guessed_letters.append(guess_letter)
    #         if verbose:
    #             print("Guessing letter: {0}".format(guess_letter))

    #         word_seen = self.get_current_word()
    #         print("current word:{}".format(word_seen))

    #         if "_" not in word_seen:
    #             print("Successfully finished game!! The word is:{}, {} tries left".format(word_seen, tries_remains))
    #             return True

    #         if guess_letter not in self.target_word:
    #             tries_remains -= 1

    #     print("# of tries exceeded!")
    #     return False

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [6]:
api = HangmanAPI(access_token="1351d29da90b3d587ad0f8b1720f69", timeout=2000)


NameError: name 'RNN_model' is not defined

In [53]:
api.start_game(practice=1)

Word: ['_', '_', '_', '_']
Successfully start a new game! Game ID: f66a2bac6bae. # of tries remaining: 6. Word: ['_', '_', '_', '_'].
Guessing letter: a
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ '}
Guessing letter: e
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 5, 'word': 'e _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 4, 'word': 'e _ _ _ '}
Guessing letter: l
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 3, 'word': 'e _ _ _ '}
Guessing letter: i
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 2, 'word': 'e _ _ _ '}
Guessing letter: t
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 1, 'word': 'e _ _ _ '}
Guessing letter: d
Sever response: {'game_id': 'f66a2bac6bae', 'status': 'ongoing', 'tries_remains': 1, 'word': 'e _ d _ '

False

In [48]:
# words=["ab","bc"]

# print(ngrams)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [55]:
[total_practice_runs_,total_recorded_runs,total_recorded_successes,total_practice_successes_] = api.my_status()
for _ in range(100):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('practice success rate so far = %.3f' % (total_practice_successes-total_practice_successes_)/(total_practice_runs-total_practice_runs_))
print(total_practice_successes-total_practice_successes_)

Word: ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
Successfully start a new game! Game ID: f33e9ad24dd7. # of tries remaining: 6. Word: ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_'].
Guessing letter: e
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ n _ e _ _ _ _ i n _ '}
Guessing letter: r
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ e _ _ _ _ i n _ '}
Guessing letter: u
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 5, 'word': 'u n _ e _ _ u _ i n _ '}
Guessing letter: g
Sever response: {'game_id': 'f33e9ad24dd7', 'status': 'ongoing', 'tries_remains': 5, 'w

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.distributed as dist
import collections
CUDA = torch.cuda.is_available()


def show_game(original_word, guesses, obscured_words_seen):
    print('Hidden word was "{}"'.format(original_word))

    for i in range(len(guesses)):
        word_seen = ''.join([chr(i + 97) if i != 26 else ' ' for i in obscured_words_seen[i].argmax(axis=1)])
        print('Guessed {} after seeing "{}"'.format(guesses[i], word_seen))

def list2tensor(arr):
    arr = np.array(arr)
    return torch.from_numpy(arr)
class Word2Batch:
    def __init__(self, model, word, lives=6):
        self.origin_word = word
        self.guessed_letter = set()  # each element should be a idx
        self.word_idx = [ord(i)-97 for i in word]
        self.remain_letters = set(self.word_idx)
        self.model = model
        self.lives_left = lives
        self.guessed_letter_each = []

        # the following is the dataset for variable to output
        self.obscured_word_seen = []  # n * 27, where n is the number of guesses
        self.prev_guessed = []  # n*26, where n is the number of guesses and each element is the normalized word idx
        self.correct_response = []  # this is the label, meaning self.prev_guess should be one of self.correct_response

    def encode_obscure_word(self):
        word = [i if i in self.guessed_letter else 26 for i in self.word_idx]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word):
            obscured_word[i, j] = 1
        return obscured_word

    def encode_prev_guess(self):
        guess = np.zeros(26, dtype=np.float32)
        for i in self.guessed_letter:
            guess[i] = 1.0
        return guess

    def encode_correct_response(self):
        response = np.zeros(26, dtype=np.float32)
        for i in self.remain_letters:
            response[i] = 1.0
        response /= response.sum()
        return response

    def game_mimic(self):
        obscured_words_seen = []
        prev_guess_seen = []
        correct_response_seen = []

        while self.lives_left > 0 and len(self.remain_letters) > 0:
            # store obscured word and previous guesses -- act as X for the label
            obscured_word = self.encode_obscure_word()
            prev_guess = self.encode_prev_guess()

            obscured_words_seen.append(obscured_word)
            prev_guess_seen.append(prev_guess)
            obscured_word = torch.from_numpy(obscured_word)
            prev_guess = torch.from_numpy(prev_guess)
            if CUDA:
                obscured_word = obscured_word.cuda()
                prev_guess = prev_guess.cuda()

            self.model.eval()
            guess = self.model(obscured_word, prev_guess)  # output of guess should be a 1 by 26 vector
            guess = torch.argmax(guess, dim=2).item()
            self.guessed_letter.add(guess)
            self.guessed_letter_each.append(chr(guess + 97))

            # store correct response -- act as label for the model
            correct_response = self.encode_correct_response()
            correct_response_seen.append(correct_response)

            # update letter remained and lives left
            if guess in self.remain_letters:  # only remove guess when the guess is correct
                self.remain_letters.remove(guess)

            if correct_response_seen[-1][guess] < 0.0000001:  # which means we made a wrong guess
                self.lives_left -= 1
        obscured_words_seen = list2tensor(obscured_words_seen)
        prev_guess_seen = list2tensor(prev_guess_seen)
        correct_response_seen = list2tensor(correct_response_seen)
        # correct_response_seen = correct_response_seen.long()
        return obscured_words_seen, prev_guess_seen, correct_response_seen

class StatefulLSTM(nn.Module):
    def __init__(self, in_size, out_size):
        super(StatefulLSTM, self).__init__()

        self.lstm = nn.LSTMCell(in_size, out_size)
        self.out_size = out_size

        self.h = None
        self.c = None

    def reset_state(self):
        self.h = None
        self.c = None

    def forward(self, x):
        batch_size = x.data.size()[0]
        if self.h is None:
            state_size = [batch_size, self.out_size]
            self.c = Variable(torch.zeros(state_size))
            self.h = Variable(torch.zeros(state_size))
            if CUDA:
                self.c = self.c.cuda()
                self.h = self.h.cuda()

        self.h, self.c = self.lstm(x, (self.h, self.c))

        return self.h


class LockedDropout(nn.Module):
    def __init__(self):
        super(LockedDropout,self).__init__()
        self.m = None

    def reset_state(self):
        self.m = None

    def forward(self, x, dropout=0.5, train=True):
        if train==False:
            return x
        if(self.m is None):
            self.m = x.data.new(x.size()).bernoulli_(1 - dropout)
        mask = Variable(self.m, requires_grad=False) / (1 - dropout)

        return mask * x


class RNN_model(nn.Module):
    def __init__(self, hidden_units=16, target_dim=26):
        super(RNN_model, self).__init__()

        # self.embedding = nn.Embedding(vocab_size,no_of_hidden_units)#,padding_idx=0)

        self.lstm1 = StatefulLSTM(27, hidden_units)
        self.bn_lstm1 = nn.BatchNorm1d(hidden_units)
        self.dropout1 = LockedDropout()
        self.fc = nn.Linear(hidden_units + 26, target_dim)


        # self.loss = nn.BCEWithLogitsLoss()

    def reset_state(self):
        self.lstm1.reset_state()
        self.dropout1.reset_state()

    def forward(self, obscure_word, prev_guess, train=True):
        if len(obscure_word.size()) < 3:
            obscure_word = obscure_word.unsqueeze(0)
        if len(prev_guess.size()) < 2:
            prev_guess = prev_guess.unsqueeze(0)

        no_of_timesteps = obscure_word.shape[0]
        batch_size = obscure_word.shape[1]
        self.reset_state()

        outputs = []
        for i in range(no_of_timesteps):
            h = self.lstm1(obscure_word[i, :, :])
            h = self.bn_lstm1(h)
            h = self.dropout1(h, dropout=0.1, train=train)

            pool = nn.MaxPool1d(batch_size)
            h = h.permute(1, 0)  # (batch_size,features,time_steps)
            h = h.unsqueeze(0)
            out = pool(h)
            out = out.squeeze(2)
            curr_prev_guess = prev_guess[i, :]
            curr_prev_guess = curr_prev_guess.unsqueeze(0)
            out = torch.cat((out, curr_prev_guess), 1)
            out = self.fc(out)
            outputs.append(out)
        outputs = torch.stack(outputs)
        return outputs

model = RNN_model()
print(model)

RNN_model(
  (lstm1): StatefulLSTM(
    (lstm): LSTMCell(27, 16)
  )
  (bn_lstm1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): LockedDropout()
  (fc): Linear(in_features=42, out_features=26, bias=True)
)


In [ ]:

import argparse
import collections
import pandas as pd
import numpy as np
from model import RNN_model
import torch

def arg_parser():
    parser = argparse.ArgumentParser(description="hangman game config")
    parser.add_argument("--train_set", type=str, default="words_250000_train.txt",
                        help="path of the train dictionary")
    parser.add_argument("--lives", type=int, default=6,
                        help="upper limit of fail guesses")
    args = parser.parse_args()
    return args


def load_model(model_path):
    model = RNN_model(target_dim=26, hidden_units=16)
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model


class HangmanGame(object):
    def __init__(self, train_set_path, model_path="model.pth", n_gram=2):
        self.guessed_letters = []
        full_dictionary_location = train_set_path
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.freq_by_length = self.init_df(self.full_dictionary)
        self.n_gram = self.init_n_gram(n_gram)
        self.current_dictionary = []
        self.history_condition = []
        self.model = load_model(model_path)

    def find_by_gram(self, all_gram, pre=None, suff=None):
        selected_gram = []
        for key, val in all_gram.items():
            if (pre is not None) and (key[0] == pre):
                selected_gram.append((key[1], val))
            if (suff is not None) and (key[1] == suff):
                selected_gram.append((key[0], val))

        res = {}
        for letter, freq in selected_gram:
            if letter not in res:
                res[letter] = freq
            else:
                res[letter] += freq
        final_res = [(key, val) for key, val in res.items()]
        return sorted(final_res, key=lambda x: x[1], reverse=True)

    def gen_n_gram(self, word, n):
        n_gram = []
        for i in range(n, len(word)+1):
            if word[i-n:i] not in n_gram:
                n_gram.append(word[i-n:i])
        return n_gram

    def init_n_gram(self, n):
        n_gram = {-1:[]}
        for word in self.full_dictionary:
            single_word_gram = self.gen_n_gram(word, n)
            if len(word) not in n_gram:
                n_gram[len(word)] = single_word_gram
            else:
                n_gram[len(word)].extend(single_word_gram)
            n_gram[-1].extend(single_word_gram)
        res = {}
        for key in n_gram.keys():
            res[key] = collections.Counter(n_gram[key])
        return res

    def freq_from_df(self, df):
        key, cnt = np.unique(df.values, return_counts=True)
        freq = [(k, val) for k, val in zip(key, cnt)]
        return sorted(freq, key=lambda x: x[1], reverse=True)

    def update_df(self, df, condition):
        """
        :param df: dataframe
        each column is one location of a word
        each row is a word
        :param condition: dictionary
        key is letter
        value is which index does this letter appear
        means we only select the words which has letter <value> at index <key>
        note that we don't select words that has letter <value> at other index
        e.g. if condition = {1:'a'}, then "app" is selected while "aha" not
        :return:
        df: updated dataframe
        """
        if len(condition) == 0:
            return df

        for letter, idx in condition.items():
            # find rows satisfy
            # 1. corresponding column == val
            # 2. all the other column != val
            query = ""
            for i in range(df.shape[1]):
                col = df.columns.values[i]
                if i in idx:
                    query += "{} == '{}' and ".format(col, letter)
                else:
                    query += "{} != '{}' and ".format(col, letter)
            query = query[:-5]
            new_df = df.query(query)
            df = new_df.copy()
            del new_df
        return df

    def init_df(self, dictionary):
        """
        use words list to generate dictionary frequency
        each key is word length
        each value is a dataframe with column is location of each length
        """
        group_by_length = collections.defaultdict(list)
        for word in dictionary:
            group_by_length[len(word)].append(word)

        res = {}
        for key in group_by_length.keys():
            word_list = group_by_length[key]
            tmp = pd.DataFrame([list(word) for word in word_list])
            tmp.columns = [chr(i + 97) for i in range(tmp.shape[1])]
            res[key] = tmp
        return res

    def gen_condition(self, word):
        tmp = {i: word[i] for i in range(len(word)) if word[i] != "_"}
        condition = {}
        for key, val in tmp.items():
            if val not in condition:
                condition[val] = [key]
            else:
                condition[val].append(key)
        return condition

    def encode_obscure_words(self, word):
        word_idx = [ord(i) - 97 if i != "_" else 26 for i in word]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word_idx):
            obscured_word[i, j] = 1
        return obscured_word

    def guess(self, word):  # word input example: "_ p p _ e "

        # divided word group by word length
        all_words = self.freq_by_length[len(word)]
        all_gram = self.n_gram[-1]
        # all_gram = self.n_gram[len(word)]

        # first guess by letter frequency in each word group
        new_condition = self.gen_condition(word)

        if len(self.history_condition) != 0 and new_condition != self.history_condition[-1]:
            self.history_condition.append(new_condition)

        all_words = self.update_df(all_words, new_condition)
        freq = self.freq_from_df(all_words)
        for i in range(len(freq)):
            if freq[i][0] not in self.guessed_letters:
                return freq[i][0]

        # if we run out of letters, use 2-gram to predict
        for i in range(len(word)):
            if word[i] == "_":  # this is where we should apply 2-gram
                if (i == 0) or (word[i-1] == "_"):
                    guess = self.find_by_gram(all_gram, pre=None, suff=word[i+1])
                elif (i == len(word) - 1) or (word[i+1] == "_"):
                    guess = self.find_by_gram(all_gram, pre=word[i-1], suff=None)
                else:
                    guess = self.find_by_gram(all_gram, pre=word[i-1], suff=word[i+1])
                break

        for i in range(len(guess)):
            if guess[i][0] not in self.guessed_letters:
                return guess[i][0]
        # if we run out of 2-gram, use LSTM model to predict
        # the benefit of LSTM model is to add more uncertainty to the prediction
        guessed_multi_hot = np.zeros(26, dtype=np.float32)
        for letter in self.guessed_letters:
            idx = ord(letter) - 97
            guessed_multi_hot[idx] = 1.0

        obscure_words = self.encode_obscure_words(word)
        obscure_words = np.asarray(obscure_words)
        guessed_multi_hot = np.asarray(guessed_multi_hot)
        obscure_words = torch.from_numpy(obscure_words)
        guessed_multi_hot = torch.from_numpy(guessed_multi_hot)
        out = self.model(obscure_words, guessed_multi_hot)
        guess = torch.argmax(out, dim=2).item()
        guess = chr(guess + 97)
        return guess

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def get_current_word(self):
        """
        combine target word and guessed letters to generate obscured word
        """
        word_seen = [letter if letter in self.guessed_letters else "_" for letter in self.target_word]
        return word_seen
